# Target Networks: The Slow Twin That Stabilizes Training

Target networks are the second key innovation that made DQN stable. Without them, DQN would chase a moving target and never converge!

## What You'll Learn

By the end of this notebook, you'll understand:
- The moving target problem (with a soccer analogy!)
- Why targets change when we update the Q-network
- Hard vs soft target updates
- Implementing target networks in PyTorch
- The impact on training stability

**Prerequisites:** Notebooks 2-3 (DQN, Experience Replay)

**Time:** ~25 minutes

---
## The Big Picture: The Soccer Analogy

```
    ┌────────────────────────────────────────────────────────────────┐
    │          THE MOVING TARGET PROBLEM: SOCCER ANALOGY            │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  Imagine you're learning to kick a soccer ball into a goal.   │
    │                                                                │
    │  NORMAL SITUATION (Stable Target):                            │
    │    🥅 The goal stays in one place                             │
    │    ⚽ You kick, miss slightly to the left                     │
    │    📊 You adjust: "Aim a bit more to the right"              │
    │    ✓ You keep improving!                                      │
    │                                                                │
    │  DQN WITHOUT TARGET NETWORK (Moving Target):                  │
    │    🥅 Every time you kick, the goal MOVES!                    │
    │    ⚽ You kick, it goes where the goal WAS                    │
    │    📊 But now the goal is somewhere else...                   │
    │    ❌ You can never hit a moving target!                      │
    │                                                                │
    │  DQN WITH TARGET NETWORK (Semi-Stable Target):                │
    │    🥅 The goal stays FIXED for 100 kicks                      │
    │    ⚽ You learn to hit that position                          │
    │    📊 Goal moves to new spot, you adapt                       │
    │    ✓ You can track the slowly-moving target!                  │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch, Rectangle, Circle, FancyArrowPatch

# Visualize the moving target problem
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Left: Stable target (normal learning)
ax1 = axes[0]
ax1.set_xlim(0, 10)
ax1.set_ylim(0, 10)
ax1.axis('off')
ax1.set_title('Normal Learning\n(Fixed Target)', fontsize=12, fontweight='bold', color='#388e3c')

# Fixed target
target_circle = Circle((7, 5), 0.8, facecolor='#c8e6c9', edgecolor='#388e3c', linewidth=3)
ax1.add_patch(target_circle)
ax1.text(7, 5, 'TARGET', ha='center', va='center', fontsize=8, fontweight='bold')

# Agent improving
for i, (x, y) in enumerate([(2, 3), (3, 4), (4.5, 4.5), (6, 5)]):
    alpha = 0.3 + 0.2 * i
    ax1.scatter(x, y, s=100, c='blue', alpha=alpha, zorder=5)
    if i < 3:
        ax1.annotate('', xy=(x+0.3, y+0.2), xytext=(x-0.2, y-0.2),
                    arrowprops=dict(arrowstyle='->', lw=1, color='blue', alpha=alpha))

ax1.text(5, 2, '✓ Converges!', ha='center', fontsize=11, color='#388e3c', fontweight='bold')
ax1.text(5, 1, 'Target stays put', ha='center', fontsize=10)

# Middle: Moving target (unstable)
ax2 = axes[1]
ax2.set_xlim(0, 10)
ax2.set_ylim(0, 10)
ax2.axis('off')
ax2.set_title('DQN Without Target Net\n(Moving Target)', fontsize=12, fontweight='bold', color='#d32f2f')

# Multiple target positions (moving)
target_positions = [(3, 7), (7, 6), (5, 3), (8, 8)]
for i, (tx, ty) in enumerate(target_positions):
    alpha = 0.2 + 0.2 * i
    circle = Circle((tx, ty), 0.5, facecolor='#ffcdd2', edgecolor='#d32f2f', 
                    linewidth=2, alpha=alpha)
    ax2.add_patch(circle)
    if i > 0:
        ax2.annotate('', xy=(tx, ty), xytext=target_positions[i-1],
                    arrowprops=dict(arrowstyle='->', lw=2, color='red', alpha=0.5,
                                   connectionstyle='arc3,rad=0.2'))

# Chasing agent (never catches up)
ax2.scatter([2, 4, 6, 7], [6, 5, 4, 7], s=80, c='blue', zorder=5)

ax2.text(5, 1.5, '❌ Never converges!', ha='center', fontsize=11, color='#d32f2f', fontweight='bold')
ax2.text(5, 0.5, 'Target moves every update', ha='center', fontsize=10)

# Right: Target network (stable enough)
ax3 = axes[2]
ax3.set_xlim(0, 10)
ax3.set_ylim(0, 10)
ax3.axis('off')
ax3.set_title('DQN With Target Net\n(Periodic Updates)', fontsize=12, fontweight='bold', color='#1976d2')

# Two fixed positions with arrows
circle1 = Circle((3, 6), 0.6, facecolor='#bbdefb', edgecolor='#1976d2', linewidth=2)
ax3.add_patch(circle1)
ax3.text(3, 6, 'T1', ha='center', va='center', fontsize=10, fontweight='bold')

circle2 = Circle((7, 5), 0.6, facecolor='#90caf9', edgecolor='#1976d2', linewidth=3)
ax3.add_patch(circle2)
ax3.text(7, 5, 'T2', ha='center', va='center', fontsize=10, fontweight='bold')

# Arrow showing periodic update
ax3.annotate('', xy=(6.3, 5.2), xytext=(3.7, 5.8),
            arrowprops=dict(arrowstyle='->', lw=2, color='#1976d2', linestyle='--'))
ax3.text(5, 6.5, 'Update every\nN steps', ha='center', fontsize=9, color='#1976d2')

# Agent converging to each
ax3.scatter([1.5, 2.2, 2.7], [5.5, 5.8, 6], s=60, c='blue', alpha=0.5)
ax3.scatter([5, 6, 6.8], [5.5, 5.3, 5.1], s=60, c='blue')

ax3.text(5, 1.5, '✓ Converges!', ha='center', fontsize=11, color='#1976d2', fontweight='bold')
ax3.text(5, 0.5, 'Target fixed between updates', ha='center', fontsize=10)

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("THE MOVING TARGET PROBLEM")
print("="*70)
print("""
In Q-learning, the target is: y = r + γ × max_a' Q(s', a'; θ)

THE PROBLEM:
  - The target uses Q(s', a'; θ)
  - When we update θ, the target CHANGES!
  - We're chasing a target that moves every time we step!

THE SOLUTION: Use TWO networks
  - Q-network θ: Updated every step (for predictions)
  - Target network θ⁻: Updated slowly (for stable targets)
  
  y = r + γ × max_a' Q(s', a'; θ⁻)  ← Uses FROZEN network!
""")
print("="*70)

---
## Why Targets Move: A Mathematical View

```
    ┌────────────────────────────────────────────────────────────────┐
    │              THE MATHEMATICS OF MOVING TARGETS                 │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  WITHOUT TARGET NETWORK:                                      │
    │                                                                │
    │  Step 1: Compute target                                       │
    │    y = r + γ × max Q(s', a'; θ)                              │
    │                            ↑                                   │
    │                    uses current θ                              │
    │                                                                │
    │  Step 2: Update network                                       │
    │    θ ← θ - α × ∇(y - Q(s, a; θ))²                            │
    │    ↑                                                          │
    │    θ changed!                                                  │
    │                                                                │
    │  Step 3: But wait...                                          │
    │    The target y now gives a DIFFERENT value                   │
    │    because it depends on the changed θ!                       │
    │                                                                │
    │  WITH TARGET NETWORK:                                         │
    │                                                                │
    │  Step 1: Compute target (using FROZEN θ⁻)                     │
    │    y = r + γ × max Q(s', a'; θ⁻)                             │
    │                            ↑                                   │
    │                    uses frozen θ⁻                              │
    │                                                                │
    │  Step 2: Update main network                                  │
    │    θ ← θ - α × ∇(y - Q(s, a; θ))²                            │
    │                                                                │
    │  Step 3: Target is STABLE!                                    │
    │    θ⁻ hasn't changed, so y stays the same                     │
    │                                                                │
    │  Periodically: Copy θ to θ⁻                                   │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Implement Q-network and target network

class QNetwork(nn.Module):
    """Simple Q-Network for demonstration."""
    
    def __init__(self, state_dim, action_dim, hidden_dim=128):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, action_dim)
        )
    
    def forward(self, x):
        return self.network(x)


# Create Q-network and target network
state_dim = 4
action_dim = 2

q_network = QNetwork(state_dim, action_dim)
target_network = QNetwork(state_dim, action_dim)

# ========================================
# KEY: Initialize target to match Q-network
# ========================================
target_network.load_state_dict(q_network.state_dict())

print("TWO NETWORKS CREATED")
print("="*60)
print("\n1. Q-NETWORK (Main Network)")
print("   - Used for action selection")
print("   - Updated EVERY training step")
print("   - Learns continuously")

print("\n2. TARGET NETWORK (Frozen Copy)")
print("   - Used for computing targets")
print("   - Updated only PERIODICALLY")
print("   - Provides stable targets")

# Verify they start identical
test_state = torch.randn(1, state_dim)
with torch.no_grad():
    q_output = q_network(test_state)
    target_output = target_network(test_state)

print(f"\nQ-network output: {q_output.numpy()[0]}")
print(f"Target output:    {target_output.numpy()[0]}")
print(f"\nIdentical at start: {torch.allclose(q_output, target_output)}")
print("="*60)

---
## Hard vs Soft Updates

There are two ways to update the target network:

```
    ┌────────────────────────────────────────────────────────────────┐
    │              HARD vs SOFT UPDATES                              │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  HARD UPDATE (Original DQN):                                  │
    │    Every N steps (e.g., N=1000):                              │
    │      θ⁻ ← θ                                                   │
    │                                                                │
    │    "Suddenly replace the target network"                      │
    │                                                                │
    │    ✓ Simple to implement                                      │
    │    ✓ Target is completely stable between copies               │
    │    ✗ Big jump when copy happens                               │
    │                                                                │
    │  SOFT UPDATE (Polyak Averaging):                              │
    │    Every step:                                                │
    │      θ⁻ ← τ × θ + (1 - τ) × θ⁻                               │
    │                                                                │
    │    Where τ is small (e.g., 0.005)                             │
    │    "Slowly blend toward the Q-network"                        │
    │                                                                │
    │    ✓ Smoother updates, no big jumps                           │
    │    ✓ Always tracking the Q-network slowly                     │
    │    ✗ Target changes slightly every step                       │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
def hard_update(target_net, q_net):
    """
    Hard Update: Completely copy weights from Q-network to target.
    
    θ⁻ ← θ
    
    Called every N steps (e.g., every 1000 training steps).
    """
    target_net.load_state_dict(q_net.state_dict())


def soft_update(target_net, q_net, tau=0.005):
    """
    Soft Update (Polyak Averaging): Slowly blend weights.
    
    θ⁻ ← τ × θ + (1 - τ) × θ⁻
    
    Called every step. tau controls the blending rate:
    - tau = 1.0: Same as hard update
    - tau = 0.005: Target moves 0.5% toward Q-network each step
    """
    for target_param, q_param in zip(target_net.parameters(), q_net.parameters()):
        # Blend: target = tau * q + (1 - tau) * target
        target_param.data.copy_(tau * q_param.data + (1 - tau) * target_param.data)


# Demonstrate the difference
print("COMPARING HARD VS SOFT UPDATES")
print("="*60)

# Get initial weight
initial_weight = q_network.network[0].weight.data[0, 0].item()
print(f"\nInitial Q-network weight: {initial_weight:.4f}")

# Modify Q-network (simulate training)
with torch.no_grad():
    q_network.network[0].weight.data[0, 0] = 5.0

new_weight = q_network.network[0].weight.data[0, 0].item()
print(f"After 'training', Q-network weight: {new_weight:.4f}")

# Apply hard update to a fresh target
target_hard = QNetwork(state_dim, action_dim)
target_hard.network[0].weight.data[0, 0] = initial_weight  # Start at initial
hard_update(target_hard, q_network)
print(f"\nHARD UPDATE result: {target_hard.network[0].weight.data[0, 0].item():.4f}")
print("  → Jumps immediately to 5.0")

# Apply soft updates to a fresh target
target_soft = QNetwork(state_dim, action_dim)
target_soft.network[0].weight.data[0, 0] = initial_weight  # Start at initial

print("\nSOFT UPDATE (tau=0.1, 10 steps):")
for step in range(10):
    soft_update(target_soft, q_network, tau=0.1)
    if step % 3 == 0 or step == 9:
        current = target_soft.network[0].weight.data[0, 0].item()
        print(f"  Step {step+1}: {current:.4f}")

print("  → Gradually approaches 5.0")
print("="*60)

In [ ]:
# Visualize hard vs soft updates
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Simulate updates over time
n_steps = 200
initial_value = 0.0
target_value = 5.0

# Hard update (every 50 steps)
ax1 = axes[0]
hard_values = []
current_hard = initial_value

for step in range(n_steps):
    if step > 0 and step % 50 == 0:
        current_hard = target_value  # Hard copy
    hard_values.append(current_hard)

ax1.plot(range(n_steps), hard_values, 'b-', linewidth=2, label='Target Network')
ax1.axhline(y=target_value, color='gray', linestyle='--', alpha=0.5, label='Q-Network')
ax1.set_xlabel('Training Step', fontsize=11)
ax1.set_ylabel('Weight Value', fontsize=11)
ax1.set_title('Hard Update (every 50 steps)', fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Mark update points
for step in [50, 100, 150]:
    ax1.axvline(x=step, color='red', linestyle=':', alpha=0.5)
    ax1.scatter([step], [target_value], c='red', s=100, zorder=5)

# Soft update
ax2 = axes[1]
soft_values = []
current_soft = initial_value
tau = 0.01

for step in range(n_steps):
    current_soft = tau * target_value + (1 - tau) * current_soft
    soft_values.append(current_soft)

ax2.plot(range(n_steps), soft_values, 'g-', linewidth=2, label='Target Network')
ax2.axhline(y=target_value, color='gray', linestyle='--', alpha=0.5, label='Q-Network')
ax2.set_xlabel('Training Step', fontsize=11)
ax2.set_ylabel('Weight Value', fontsize=11)
ax2.set_title(f'Soft Update (τ = {tau})', fontsize=12, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nHard Update: Sudden jumps every N steps")
print("Soft Update: Smooth, gradual approach")
print("\nSoft updates are often preferred for smoother training!")

---
## The Impact on Training Stability

Let's visualize why target networks help:

In [ ]:
# Simulate TD targets with and without target network
np.random.seed(42)

n_updates = 100
true_value = 5.0
gamma = 0.99

# Without target network
q_value_no_target = 0.0
q_history_no_target = [q_value_no_target]
target_history_no_target = []

for _ in range(n_updates):
    reward = np.random.randn() * 0.5
    # Target uses same Q-value (moving target!)
    target = reward + gamma * q_value_no_target
    target_history_no_target.append(target)
    # Update toward target
    q_value_no_target += 0.1 * (target - q_value_no_target) + np.random.randn() * 0.3
    q_history_no_target.append(q_value_no_target)

# With target network (hard update every 20 steps)
q_value_with_target = 0.0
target_q = 0.0  # Target network Q-value
q_history_with_target = [q_value_with_target]
target_history_with_target = []

for step in range(n_updates):
    reward = np.random.randn() * 0.5
    # Target uses FROZEN target Q-value
    target = reward + gamma * target_q
    target_history_with_target.append(target)
    # Update toward target
    q_value_with_target += 0.1 * (target - q_value_with_target) + np.random.randn() * 0.3
    q_history_with_target.append(q_value_with_target)
    
    # Hard update every 20 steps
    if (step + 1) % 20 == 0:
        target_q = q_value_with_target

# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax1 = axes[0]
ax1.plot(q_history_no_target, 'r-', linewidth=2, label='Q-value')
ax1.plot(target_history_no_target, 'r--', alpha=0.5, label='Target')
ax1.set_xlabel('Training Step', fontsize=11)
ax1.set_ylabel('Value', fontsize=11)
ax1.set_title('WITHOUT Target Network\n(Targets move with Q)', fontsize=12, fontweight='bold', color='#d32f2f')
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2 = axes[1]
ax2.plot(q_history_with_target, 'g-', linewidth=2, label='Q-value')
ax2.plot(target_history_with_target, 'b--', alpha=0.5, label='Target (from target net)')
for step in range(20, n_updates, 20):
    ax2.axvline(x=step, color='gray', linestyle=':', alpha=0.3)
ax2.set_xlabel('Training Step', fontsize=11)
ax2.set_ylabel('Value', fontsize=11)
ax2.set_title('WITH Target Network\n(Targets fixed between copies)', fontsize=12, fontweight='bold', color='#388e3c')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nObservations:")
print("- Without target network: More erratic, targets chase Q-values")
print("- With target network: Smoother, targets provide stable reference")

---
## Putting It Together: DQN Target Computation

Here's how targets are computed in DQN:

In [ ]:
def compute_dqn_targets(q_network, target_network, 
                        rewards, next_states, dones, gamma=0.99):
    """
    Compute DQN targets using the target network.
    
    target = r + γ × max_a' Q(s', a'; θ⁻)
                               ↑
                       Uses TARGET network!
    """
    with torch.no_grad():
        # Use TARGET network for computing max Q(s', a')
        next_q_values = target_network(next_states)
        max_next_q = next_q_values.max(dim=1)[0]
        
        # Compute targets
        # Note: (1 - dones) zeros out future value for terminal states
        targets = rewards + gamma * max_next_q * (1 - dones)
    
    return targets


# Demonstrate
print("TARGET COMPUTATION IN DQN")
print("="*60)

# Reset networks
q_network = QNetwork(state_dim, action_dim)
target_network = QNetwork(state_dim, action_dim)
target_network.load_state_dict(q_network.state_dict())

# Sample batch
batch_size = 4
next_states = torch.randn(batch_size, state_dim)
rewards = torch.tensor([1.0, 0.0, 1.0, 10.0])
dones = torch.tensor([0.0, 0.0, 0.0, 1.0])  # Last one is terminal

# Compute targets
targets = compute_dqn_targets(q_network, target_network, rewards, next_states, dones)

print("\nBatch Information:")
print(f"  Rewards: {rewards.tolist()}")
print(f"  Dones:   {dones.tolist()}")

print(f"\nComputed Targets: {targets.tolist()}")
print("\nNote: Terminal state (done=1) has target = reward only")
print(f"  (no future value since episode ended)")
print("="*60)

---
## Summary: Key Takeaways

### The Problem

| Issue | Description |
|-------|-------------|
| **Moving Targets** | Target depends on Q-network that's being updated |
| **Instability** | Updates chase a constantly-changing goal |
| **Non-convergence** | Network oscillates instead of improving |

### The Solution

| Component | Purpose |
|-----------|--------|
| **Target Network (θ⁻)** | Provides stable targets for training |
| **Periodic Updates** | Slowly sync with Q-network |

### Update Methods

| Method | Formula | When to Use |
|--------|---------|-------------|
| **Hard** | θ⁻ ← θ (every N steps) | Original DQN |
| **Soft** | θ⁻ ← τθ + (1-τ)θ⁻ (every step) | DDPG, SAC |

---
## Test Your Understanding

**1. Why do targets "move" in Q-learning without a target network?**
<details>
<summary>Click to reveal answer</summary>
The target y = r + γ × max Q(s', a'; θ) depends on the Q-network weights θ. When we update θ to minimize the loss, the Q-values for next states change, which changes the target. So both the prediction and target change together!
</details>

**2. How does a target network solve this problem?**
<details>
<summary>Click to reveal answer</summary>
The target network θ⁻ is a separate copy that's updated slowly (or periodically). Targets are computed using θ⁻, which stays fixed while the main network θ updates. This provides a stable goal to train toward.
</details>

**3. What's the difference between hard and soft updates?**
<details>
<summary>Click to reveal answer</summary>
Hard update: θ⁻ ← θ every N steps (complete copy)
Soft update: θ⁻ ← τθ + (1-τ)θ⁻ every step (gradual blend)

Hard updates cause sudden changes every N steps. Soft updates are smoother but targets change slightly every step.
</details>

**4. What happens to the target when an episode ends (done=True)?**
<details>
<summary>Click to reveal answer</summary>
For terminal states, the target is just the reward: y = r (no γ × max Q term). There's no future value because the episode has ended. We multiply by (1 - done) to zero out the future value term.
</details>

---
## What's Next?

You now understand both key innovations of DQN:
1. **Experience Replay** - Breaks correlations
2. **Target Networks** - Stabilizes targets

In the next notebook, we'll explore improvements to DQN!

**Continue to:** [Notebook 5: DQN Improvements](05_dqn_improvements.ipynb)

---

*Target networks: "Give the learner a stable goal to chase."*